In [1]:
# This is a Basic Chatbot that features intent detection and response generation

In [2]:
# Imports
import json
import numpy as np
# slkearn text preprocessing
from sklearn.preprocessing import LabelEncoder

# Tokenizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D

# Saving model and data
import pickle

2023-05-14 18:02:40.392543: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-14 18:02:40.452631: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-14 18:02:46.915282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Loading json data
with open('intent.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []

In [4]:
# Extracting the required data
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

print(training_labels)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'about', 'about', 'about', 'name', 'name', 'name', 'help', 'help', 'help', 'help', 'help', 'help', 'help', 'createaccount', 'createaccount', 'createaccount', 'createaccount', 'createaccount', 'complaint', 'complaint', 'complaint']


In [5]:
# Using "LabelEncoder" to convert the target labels into a model understandable form
lbl = LabelEncoder()
lbl.fit(training_labels)
training_labels = lbl.transform(training_labels)
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [6]:
# Now lets vectorize our text data corpus by using the "Tokenizer" class and it allows us to limit our vocabulary
# size up to some defined number. When we use this class for our text preprocessing, by default it will remove all
# punctuations, turning the text into space seperated sequences of words. These sequences are then split into lists of tokens.
# They will then be indexed or vectorized.
# We can also add "oov_token" which is the value of "out of token" to deal with our vocabulary words(tokens) at inference time.

In [7]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
# the "pad_sequence" method is used to make all the training text sequences into the same size.
padded_sequences = pad_sequences(sequences, truncating='post', maxlen = max_len)

In [8]:
# Model Training
model = Sequential()

# Below are the layers of our model
model.add(Embedding(vocab_size, embedding_dim, input_length = max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [9]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 13ms/step - loss: 2.0798 - accuracy: 0.1212
Epoch 2/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0785 - accuracy: 0.1515
Epoch 3/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0782 - accuracy: 0.2121
Epoch 4/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0776 - accuracy: 0.2121
Epoch 5/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0770 - accuracy: 0.2121
Epoch 6/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0764 - accuracy: 0.2424
Epoch 7/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0761 - accuracy: 0.3030
Epoch 8/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0757 - accuracy: 0.1818
Epoch 9/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0753 - accuracy: 0.0909
Epoch 10/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0751 - accuracy: 0.0909
Epoch 11/500
2/2 [==========

In [10]:
# After training its important to save all the required files

In [11]:
# So we save our model
model.save("chat_model")

# To save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl, ecn_file, protocol = pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets
